In [1]:
import pandas as pd
from unidecode import unidecode
from difflib import get_close_matches

In [2]:

# --- Cargar datos  ---
tabla1 = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/data_colegio/EPC6.csv")
tabla2 = pd.read_csv("C:/Users/juana/Downloads/ID GEO ESPAÑA - Hoja 1.csv")

# --- Preprocesar textos: quitar acentos y pasar todo a minúsculas ---
def limpiar(texto):
    return unidecode(str(texto)).lower().strip()

tabla1["Comunidad/Provincia_limpia"] = tabla1["Comunidad/Provincia"].apply(limpiar)
tabla2["Provincia_limpia"] = tabla2["Provincia"].apply(limpiar)
tabla2["CA_limpia"] = tabla2["Comunidad Autónoma"].apply(limpiar)

# --- Diccionarios auxiliares ---
dicc_provincias = dict(zip(tabla2["Provincia_limpia"], tabla2["ID GEO"]))
dicc_provincia_a_ca = dict(zip(tabla2["Provincia_limpia"], tabla2["CA_limpia"]))
dicc_ca = {v: k for k, v in zip(tabla2["CA_limpia"], tabla2["Comunidad Autónoma"])}  # limpio→original

# --- Función que busca coincidencia ---
def obtener_valor(valor):
    # 1) Intentamos buscar provincia
    posibles = get_close_matches(valor, dicc_provincias.keys(), n=1, cutoff=0.8)
    if posibles:
        return dicc_provincias[posibles[0]]  # Devuelve ID GEO

    # 2) Si no es provincia, probamos si es CA
    posibles_ca = get_close_matches(valor, dicc_ca.keys(), n=1, cutoff=0.8)
    if posibles_ca:
        return dicc_ca[posibles_ca[0]]  # Devuelve nombre de la CA (original con tildes)

    # 3) Si no coincide con nada, devolvemos "DESCONOCIDO"
    return "DESCONOCIDO"

# --- Crear nueva columna al inicio ---
tabla1.insert(0, "ID GEO / CA", tabla1["Comunidad/Provincia_limpia"].apply(obtener_valor))

# --- Limpiar columnas auxiliares ---
tabla1 = tabla1.drop(columns=["Comunidad/Provincia_limpia"])

# --- Resultado ---
print(tabla1.head())
df=tabla1


# Función para limpiar nombres de columnas
def limpiar_columna(col):
    col = unidecode(col)             # quita acentos
    col = col.lower()                # pasa a minúsculas
    col = col.replace(" ", "_")      # espacios por _
    col = col.replace("/", "_")      # / por _
    col = col.replace("-", "_")      # - por _
    col = ''.join(c for c in col if c.isalnum() or c == "_")  # solo letras/números/_
    return col

# Aplicar a todas las columnas
df.columns = [limpiar_columna(c) for c in df.columns]


# Eliminar columnas que tengan todos los valores vacíos (NaN o "")
df = df.dropna(axis=1, how="all")           # elimina si todo es NaN
df = df.loc[:, (df != "").any(axis=0)]      # elimina si todo es cadena vacía



   ID GEO / CA  Unnamed: 0 Comunidad/Provincia Total A_Hasta 4 A_Entre 5 y 8  \
0  DESCONOCIDO           0           Andalucía   120        27            20   
1          104           1             Almería     8         1             2   
2          111           2               Cádiz    11         4             2   
3          114           3             Córdoba    12         6             0   
4          118           4             Granada     6         2             2   

  A_Entre 9 y 12 A_Entre 13 y 16 A_Entre 17 y 19 A_20 o más      Filename  
0             27              12              11         23  2021_1T.xlsx  
1              2               2               0          1  2021_1T.xlsx  
2              3               0               1          1  2021_1T.xlsx  
3              1               0               2          3  2021_1T.xlsx  
4              2               0               0          0  2021_1T.xlsx  


In [3]:
print("Columnas después de limpiar:", df.columns.tolist())

Columnas después de limpiar: ['id_geo___ca', 'unnamed_0', 'comunidad_provincia', 'total', 'a_hasta_4', 'a_entre_5_y_8', 'a_entre_9_y_12', 'a_entre_13_y_16', 'a_entre_17_y_19', 'a_20_o_mas', 'filename']


In [4]:
# Eliminar la columna de índice si se importó como parte del DataFrame
df = df.reset_index(drop=True)

# Eliminar columna llamada 'unnamed_0' si existe
df = df.drop(columns=['unnamed_0'], errors='ignore')
df
# Lista de columnas a convertir a numerico
cols_a_convertir = ['total', 'a_hasta_4', 'a_entre_5_y_8', 'a_entre_9_y_12', 'a_entre_13_y_16', 'a_entre_17_y_19', 'a_20_o_mas']

# Convertir a numérico, los errores se convierten en NaN
for col in cols_a_convertir:
    df[col] = pd.to_numeric(df[col], errors="coerce")

print("Columnas convertidas a numéricas correctamente.")


# Cantidad de vacíos y porcentaje por columna
vacios = df.isna().sum()  # cuenta NaN por columna
porcentaje = (vacios / len(df) * 100).round(2)  # calcula el % respecto al total de filas

# Crear un DataFrame resumido
resumen_vacios = pd.DataFrame({
    "Columnas": df.columns,
    "Cantidad vacíos": vacios.values,
    "Porcentaje vacíos": porcentaje.values
})

print(resumen_vacios)

# Filtrar filas donde id_geo sea "DESCONOCIDO" (ignorando mayúsculas/minúsculas)
desconocidos = df[df['id_geo___ca'].str.upper() == 'DESCONOCIDO']

# Contar la cantidad de entradas por provincia dentro de desconocidos
resumen_provincias = desconocidos['comunidad_provincia'].value_counts().sort_values(ascending=False)

print(resumen_provincias)



# Encuentra el índice de las filas a eliminar
indices = df[df['id_geo___ca'] == 'DESCONOCIDO'].index

# Elimina esas filas
df.drop(indices, inplace=True)

# Cambiar nombre de la columna
df = df.rename(columns={"filename": "periodo"})

# Quitar todo lo que hay después del punto (.) en cada valor de la columna Periodo
df["periodo"] = df["periodo"].str.split(".").str[0]

df[["año", "trimestre"]] = df["periodo"].str.split("_", expand=True)
df["trimestre"] = df["trimestre"].str.extract(r'(\d+)')  # Extrae solo el número


df = df.drop(columns=["periodo"])
df['id_geo___ca'].value_counts()



Columnas convertidas a numéricas correctamente.
              Columnas  Cantidad vacíos  Porcentaje vacíos
0          id_geo___ca                0               0.00
1  comunidad_provincia                0               0.00
2                total                2               0.18
3            a_hasta_4                2               0.18
4        a_entre_5_y_8                2               0.18
5       a_entre_9_y_12                2               0.18
6      a_entre_13_y_16                2               0.18
7      a_entre_17_y_19                2               0.18
8           a_20_o_mas                2               0.18
9             filename                0               0.00
comunidad_provincia
Andalucía                     18
Las Palmas de Gran Canaria    18
Cataluña                      18
Alicante                      18
Castellón                     18
A Coruña                      18
Pais Vasco                    18
La Rioja                      18
Ceuta y Melilla    

id_geo___ca
639                     36
734                     36
943                     36
104                     18
816                     18
819                     18
845                     18
908                     18
917                     18
925                     18
comunitat valenciana    18
extremadura             18
1106                    18
1110                    18
galicia                 18
802                     18
1227                    18
1232                    18
1236                    18
1328                    18
1430                    18
1531                    18
1601                    18
1648                    18
1620                    18
1851                    18
813                     18
749                     18
castilla-la mancha      18
333                     18
114                     18
118                     18
121                     18
123                     18
129                     18
141                     18
222             

In [5]:
import os


# Filtrar solo las entradas NO numéricas
df_non_numeric = df[pd.to_numeric(df['id_geo___ca'], errors='coerce').isna()]

# Revisar valores únicos
print(df_non_numeric['id_geo___ca'].value_counts())


id_geo___ca
castilla y leon         18
castilla-la mancha      18
comunitat valenciana    18
extremadura             18
galicia                 18
Name: count, dtype: int64


In [6]:
import os


# Filtrar solo las entradas NO numéricas
df_non_numeric = df[pd.to_numeric(df['id_geo___ca'], errors='coerce').isna()]

# Revisar valores únicos
print(df_non_numeric['id_geo___ca'].value_counts())

df = df[pd.to_numeric(df['id_geo___ca'], errors='coerce').notna()]
df_non_numeric = df[pd.to_numeric(df['id_geo___ca'], errors='coerce').isna()]
print(df_non_numeric['id_geo___ca'].value_counts())

id_geo___ca
castilla y leon         18
castilla-la mancha      18
comunitat valenciana    18
extremadura             18
galicia                 18
Name: count, dtype: int64
Series([], Name: count, dtype: int64)


In [7]:


# Definir ruta destino
ruta_destino = r"C:\Users\juana\Desktop\liquideuda_project\datasets_clean"

# Guardar CSV en la ruta destino
ruta_csv = os.path.join(ruta_destino, "EPC6_clean.csv")
df.to_csv(ruta_csv, index=False, encoding="utf-8-sig")

print(f"Archivo guardado en: {ruta_csv}")

Archivo guardado en: C:\Users\juana\Desktop\liquideuda_project\datasets_clean\EPC6_clean.csv


In [8]:
df.columns

Index(['id_geo___ca', 'comunidad_provincia', 'total', 'a_hasta_4',
       'a_entre_5_y_8', 'a_entre_9_y_12', 'a_entre_13_y_16', 'a_entre_17_y_19',
       'a_20_o_mas', 'año', 'trimestre'],
      dtype='object')